In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

### Utility functions

In [2]:
def print_classification_report(clf, X_test, y,test):
    y_pred = clf.predict(X_test)
    print(classification_report(y_test,y_pred))

# Dataset completo 
Concatenazione delle feature

In [3]:
df = pd.read_csv("dataset_eeg_nseg.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [4]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1      0.000056  0.000210  0.000578  0.001068  0.001386   
                2      0.000073  0.000045  0.000349  0.000667  0.000553   
                3      0.000053  0.000243  0.000697  0.000991  0.000822   
                4      0.000032  0.000114  0.000367  0.000646  0.000741   
                5      0.000037  0.000115  0.000549  0.000856  0.000560   
...                         ...       ...       ...       ...       ...   
9       3       20     0.000005  0.000028  0.000144  0.000136  0.000052   
                21     0.000012  0.000020  0.000093  0.000228  0.000360   
                22     0.000006  0.000026  0.000080  0.000192  0.000179   
                23     0.000035  0.000116  0.000271  0.000244  0.000255   
                24     0.000016  0.000144  0.000096  0.000285  0.000275   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1      0.001648  0.000803  0.000437  0.000751  0.000383  ...   
                2      0.000305  0.000502  0.000579  0.000626  0.000521  ...   
                3      0.000288  0.000170  0.000315  0.000519  0.000392  ...   
                4      0.000520  0.000351  0.000349  0.000443  0.000537  ...   
                5      0.000430  0.000418  0.000234  0.000129  0.000227  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     0.000212  0.000210  0.000047  0.000127  0.000165  ...   
                21     0.000446  0.000264  0.000091  0.000087  0.000094  ...   
                22     0.000068  0.000094  0.000052  0.000129  0.000117  ...   
                23     0.000180  0.000203  0.000097  0.000168  0.000173  ...   
                24     0.000113  0.000217  0.000480  0.000640  0.000708  ...   

                           5502      5503      5504      5505      5506  \
id_user session video                                                     
10      1       1      0.680894  0.677372  0.678587  0.680154  0.679000   
                2      0.681449  0.680814  0.681055  0.679858  0.679219   
                3      0.680556  0.679891  0.680421  0.680982  0.680714   
                4      0.681528  0.681170  0.682441  0.681751  0.681922   
                5      0.681270  0.679830  0.681256  0.682198  0.681864   
...                         ...       ...       ...       ...       ...   
9       3       20     0.683626  0.684026  0.683992  0.683872  0.683737   
                21     0.683333  0.683626  0.683485  0.683547  0.683689   
                22     0.683917  0.683562  0.683447  0.683692  0.683361   
                23     0.683632  0.683680  0.683738  0.683844  0.683406   
                24     0.683428  0.683518  0.683481  0.683462  0.683615   

                           5507      5508      5509      5510  emotion  
id_user session video                                                   
10      1       1      0.679539  0.679375  0.678674  0.680591        1  
                2      0.680615  0.676147  0.680797  0.679779        2  
                3      0.680317  0.680449  0.680310  0.679916        3  
                4      0.682117  0.682258  0.681055  0.681194        0  
                5      0.680023  0.680439  0.680000  0.679143        2  
...                         ...       ...       ...       ...      ...  
9       3       20     0.683521  0.683858  0.683820  0.683911        0  
                21     0.683664  0.683373  0.683693  0.684037        2  
                22     0.683313  0.683518  0.684135  0.684174        0  
                23     0.683448  0.683663  0.683562  0.683567        1  
                24     0.684293  0.684325  0.684048  0.683871        0  

[1080 rows x 5512 columns]

## Split train e test
### Subject Dependent
16 trials train e 8 test
2 testing group for labels 

In [5]:
def subject_dependent_split(df,session):
    df = df.reset_index()
    np.random.seed(75)
    test_videos = np.random.choice(np.arange(1, 25), replace=False, size=(8))
    df_sess = df.loc[df['session'] == session]
    X_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    X_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    return X_train, X_test, y_train, y_test

In [6]:
scores_test = []
scores_training = []
clfSD = GradientBoostingClassifier(n_estimators=80, learning_rate=0.001, max_depth=5,random_state=104)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [7]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.22499999999999998
TRAIN: 0.6888888888888888


In [8]:
scores_test = []
scores_training = []
rfSD = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=42)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    rfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(rfSD.score(X_testSD, y_testSD))
    scores_training.append(rfSD.score(X_trainSD, y_trainSD))

In [9]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.26944444444444443
TRAIN: 0.9791666666666666


### Subject Indipendent
Leave One Out Cross Validation sugli utenti

In [10]:
def subject_independent_split(df,session):
    df = df.reset_index()
    df_sess = df.loc[df['session'] == session]
    groups = df_sess['id_user']
    X = df_sess.set_index(['id_user','session','video']).drop('emotion',axis=1)
    y = df_sess.set_index(['id_user','session','video']).emotion
    return X, y, groups

In [11]:
logo = LeaveOneGroupOut()

In [12]:
scores_test = []
gbc_SI = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_gbc.scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  8.3min finished


AttributeError: 'RandomForestClassifier' object has no attribute 'scores'

In [ ]:
print("TEST:", np.array(scores_test).mean())

In [ ]:
scores_test = []
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    scores_rfc = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores_rfc.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())

### Subject-Biased Experiment
train_test_split

In [ ]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [ ]:
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=3, random_state=42).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", clf.score(X_testSB, y_testSB))
print("TEST", clf.score(X_trainSB, y_trainSB))

In [ ]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=22).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", rfc.score(X_testSB, y_testSB))
print("TEST", rfc.score(X_trainSB, y_trainSB))

# Esecuzione su singole feature 
Dataset con singola feature tra PSD, DE, ASM, DAMS
## PSD

In [ ]:
df = pd.read_csv("dataset_eeg_psds.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [ ]:
df

### Subject Dependent

In [ ]:
scores_test = []
scores_training = []
clfSD = GradientBoostingClassifier(n_estimators=80, learning_rate=0.001, max_depth=5,random_state=104)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [ ]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

In [ ]:
scores_test = []
scores_training = []
rfSD = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=42)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    rfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(rfSD.score(X_testSD, y_testSD))
    scores_training.append(rfSD.score(X_trainSD, y_trainSD))

In [ ]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

### Subject indipendent

In [ ]:
scores_test = []
gbc_SI = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())

In [ ]:
scores_test = []
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    scores_rfc = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores_rfc.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())

### Subject Biased

In [ ]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [ ]:
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=3, random_state=42).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", clf.score(X_testSB, y_testSB))
print("TEST", clf.score(X_trainSB, y_trainSB))

In [ ]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=22).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", rfc.score(X_testSB, y_testSB))
print("TEST", rfc.score(X_trainSB, y_trainSB))

## DE

In [ ]:
df = pd.read_csv("dataset_eeg_des.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [ ]:
df

### Subject Dependent

In [ ]:
scores_test = []
scores_training = []
clfSD = GradientBoostingClassifier(n_estimators=80, learning_rate=0.001, max_depth=5,random_state=104)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [ ]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

In [ ]:
scores_test = []
scores_training = []
rfSD = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=42)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    rfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(rfSD.score(X_testSD, y_testSD))
    scores_training.append(rfSD.score(X_trainSD, y_trainSD))

In [ ]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

### Subject Independent

In [ ]:
scores_test = []
gbc_SI = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())

In [ ]:
scores_test = []
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    scores_rfc = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores_rfc.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())

### Subject-Biased

In [ ]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [ ]:
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=3, random_state=42).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", clf.score(X_testSB, y_testSB))
print("TEST", clf.score(X_trainSB, y_trainSB))

In [ ]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=22).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", rfc.score(X_testSB, y_testSB))
print("TEST", rfc.score(X_trainSB, y_trainSB))

## ASM

In [ ]:
df = pd.read_csv("dataset_eeg_asms.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [ ]:
df

### Subject Dependent

In [ ]:
scores_test = []
scores_training = []
clfSD = GradientBoostingClassifier(n_estimators=80, learning_rate=0.001, max_depth=5,random_state=104)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [ ]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

In [ ]:
scores_test = []
scores_training = []
rfSD = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=42)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    rfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(rfSD.score(X_testSD, y_testSD))
    scores_training.append(rfSD.score(X_trainSD, y_trainSD))

In [ ]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

### Subject indipendent

In [ ]:
scores_test = []
gbc_SI = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())

In [ ]:
scores_test = []
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    scores_rfc = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores_rfc.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())

### Subject Biased

In [ ]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [ ]:
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=3, random_state=42).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", clf.score(X_testSB, y_testSB))
print("TEST", clf.score(X_trainSB, y_trainSB))

In [ ]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=22).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", rfc.score(X_testSB, y_testSB))
print("TEST", rfc.score(X_trainSB, y_trainSB))

## DASM

In [ ]:
df = pd.read_csv("dataset_eeg_dasms.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [ ]:
df

### Subject Dependent

In [ ]:
scores_test = []
scores_training = []
clfSD = GradientBoostingClassifier(n_estimators=80, learning_rate=0.001, max_depth=5,random_state=104)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [ ]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

In [ ]:
scores_test = []
scores_training = []
rfSD = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=42)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    rfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(rfSD.score(X_testSD, y_testSD))
    scores_training.append(rfSD.score(X_trainSD, y_trainSD))

In [ ]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

### Subject indipendent

In [ ]:
scores_test = []
gbc_SI = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())

In [ ]:
scores_test = []
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    scores_rfc = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores_rfc.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())

### Subject Biased

In [ ]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [ ]:
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=3, random_state=42).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", clf.score(X_testSB, y_testSB))
print("TEST", clf.score(X_trainSB, y_trainSB))

In [ ]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=22).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", rfc.score(X_testSB, y_testSB))
print("TEST", rfc.score(X_trainSB, y_trainSB))